# NLP基礎 (3): サブワード分割 (Sub-word Tokenization) - BPE, WordPiece, SentencePiece

このノートブックでは、現代の自然言語処理 (NLP) モデル、特に大規模言語モデルにおいて不可欠な技術となっている**サブワード分割 (Sub-word Tokenization)** について学びます。
単語ベースのトークナイゼーションが抱える語彙爆発や未知語の問題を解決するために、Byte Pair Encoding (BPE)、WordPiece、SentencePieceといった代表的なサブワード分割アルゴリズムのアイデアと仕組みを解説します。
BPEについては、PythonとNumPyを使って主要な処理を簡易的に実装し、SentencePieceについてはライブラリの基本的な使い方を紹介します。

**参考論文:**
*   Sennrich, R., Haddow, B., & Birch, A. (2016). Neural machine translation of rare words with subword units. In *Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers)* (pp. 1715-1725). (BPEのNLPへの応用)
*   Schuster, M., & Nakajima, K. (2012). Japanese and korean voice search. In *2012 ieee international conference on acoustics, speech and signal processing (icassp)* (pp. 5149-5152). IEEE. (WordPieceの初期の言及の一つ)
*   Wu, Y., Schuster, M., Chen, Z., Le, Q. V., Norouzi, M., Macherey, W., ... & Dean, J. (2016). Google's neural machine translation system: Bridging the gap between human and machine translation. *arXiv preprint arXiv:1609.08144*. (GNMTでWordPieceを使用)
*   Kudo, T., & Richardson, J. (2018). Sentencepiece: A simple and language independent subword tokenizer and detokenizer for neural text processing. In *Proceedings of the 2018 conference on empirical methods in natural language processing: system demonstrations* (pp. 66-71). (SentencePiece提案論文)

**このノートブックで学ぶこと:**
1.  サブワード分割の必要性：語彙爆発と未知語問題。
2.  Byte Pair Encoding (BPE) のアルゴリズムと簡易実装。
3.  WordPieceの基本的なアイデアとBPEとの違い。
4.  SentencePieceの設計思想、特徴、および基本的な使い方。
5.  各サブワード分割手法の比較。

**前提知識:**
*   テキスト前処理（トークナイゼーションなど）の基本的な理解（NLP基礎(1)のノートブック）。
*   Pythonの基本的なデータ構造（辞書、リスト、セット）とNumPyの操作。

## 1. 必要なライブラリのインポート

In [22]:
import numpy as np
import re
from collections import Counter, defaultdict # defaultdictはBPEの実装に便利
import heapq # BPEで最も頻繁なペアを見つけるのに使える（今回はCounterで代用）

!pip install sentencepiece
import sentencepiece as spm

## 2. サブワード分割の必要性

前のノートブックで学んだ単語埋め込み (Word Embeddings) は、単語を単位としてベクトル表現を獲得しました。しかし、このアプローチにはいくつかの課題があります。

*   **語彙爆発 (Vocabulary Explosion):**
    *   言語には非常に多くの単語が存在し（特に形態的に豊かな言語や、新しい単語が頻繁に生まれるドメイン）、全ての単語を語彙に含めようとすると、語彙サイズが非常に大きくなります。
    *   これは、モデルのパラメータ数増加（特に埋め込み層）、メモリ使用量の増大、計算コストの増加に繋がります。
*   **未知語 (Out-of-Vocabulary, OOV) 問題:**
    *   訓練データに出現しなかった単語（未知語）は、テスト時や実際の応用時に問題となります。これらの単語は通常、特別な`<UNK>`トークンに置き換えられますが、それでは単語固有の情報が失われてしまいます。
    *   特に、固有名詞、専門用語、タイプミス、新語などは未知語になりやすいです。

**サブワード分割**は、これらの問題を解決するために、単語をより小さな単位（サブワード、例: "unfortunately" -> "un", "fortunate", "ly" や "unfortunate", "ly"）に分割するアプローチです。

**サブワード分割の利点:**
*   **語彙サイズの制御:** 頻繁に出現する単語はそのまま保持し、低頻度語や未知語はサブワードの組み合わせで表現することで、固定サイズの語彙で多くの単語をカバーできます。
*   **未知語への対応:** 未知語であっても、既知のサブワードの組み合わせで表現できる可能性が高まります（例: "jetliner" が未知でも "jet" と "liner" が既知なら表現可能）。
*   **形態情報の活用:** "un-", "-ly", "-ing" のような接辞がサブワードとして学習されれば、単語の形態的な構造や意味を捉えるのに役立ちます。
*   **データスパースネスの緩和:** 単語よりもサブワードの方がコーパス中での出現頻度が高くなるため、より頑健な統計的学習が期待できます。

## 3. Byte Pair Encoding (BPE)

BPEは、元々はデータ圧縮アルゴリズムとして提案されましたが、Sennrichら (2016) によってニューラル機械翻訳のためのサブワード分割手法として導入され、広く使われるようになりました。

### 3.1 BPEアルゴリズムのステップ

1.  **初期化:**
    *   コーパス内の全単語を文字のシーケンスに分割し、初期の語彙を全てのユニークな文字（バイト）の集合とします。各単語の終わりには特別な終端記号（例: `</w>`）を付加することが多いです。
    *   各単語の出現頻度も考慮します。
2.  **繰り返し処理 (指定されたマージ回数または語彙サイズに達するまで):**  
    a.  現在の単語分割状態において、コーパス全体で最も頻繁に出現する隣接するサブワード（または文字）のペアを見つけます。  
    b.  その最も頻繁なペアを新しい1つのサブワードとしてマージし、語彙に追加します。  
    c.  コーパス中の該当するペアを全て新しいサブワードで置き換えます。  

このプロセスを繰り返すことで、頻繁に出現する文字の組み合わせが徐々に長いサブワードとして学習されていきます。最終的な語彙には、個々の文字、学習されたサブワード、そして頻繁な単語全体が含まれることになります。

### 3.2 PythonによるBPEの簡易実装 (学習とセグメンテーション)

ここでは、BPEの核となるマージ処理のロジックを簡易的に実装してみます。

**学習フェーズ:**
1.  単語を文字に分割し、頻度をカウント。
2.  最も頻繁な隣接ペアを見つける。
3.  そのペアをマージし、単語の表現を更新。
4.  2-3を繰り返す。

**セグメンテーションフェーズ:**
学習済みのマージ操作（優先順位の高いものから）を、新しい単語に適用していく。

In [23]:
def get_word_char_freqs(corpus_words_with_freqs):
    '''単語を文字シーケンスに分割し，終端記号を付加，頻度も保持
    例) 'apple': 5 -> 'a p p l e </w>': 5
    '''
    word_char_freqs = defaultdict(int)
    for word, freq in corpus_words_with_freqs.items():
        # 単語の終わりを示す特別な記号を追加
        # 文字間にスペースを入れて，個々の文字をトークンとして扱う
        spaced_word = ' '.join(list(word)) + ' </w>'
        word_char_freqs[spaced_word] += freq
    return word_char_freqs

def get_stats(word_char_freqs):
    '''現在の単語分割における隣接ペアの頻度を計算'''
    pairs = defaultdict(int)
    for word_chars, freq in word_char_freqs.items():
        symbols = word_chars.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq # ペアの出現回数に単語の頻度を加算
    return pairs

def merge_vocab(pair_to_merge, v_in):
    '''語彙内の指定されたペアをマージする'''
    v_out = {}
    bigram = re.escape(' '.join(pair_to_merge))
    merged_symbol = ''.join(pair_to_merge)  # ペアを結合して新しいシンボルを作成

    # 正規表現で、スペースで区切られたペアを、スペースなしの結合シンボルに置き換える
    # 例: "t e s t </w>" でペア ("e", "s") をマージする場合、 "t es t </w>" になる

    for word_chars, freq in v_in.items():
        symbols = word_chars.split()
        j = 0
        new_symbols = []
        while j < len(symbols):
            if j < len(symbols)-1 and (symbols[j], symbols[j+1]) == pair_to_merge:
                new_symbols.append(merged_symbol)
                j += 2
            else:
                new_symbols.append(symbols[j])
                j += 1
        v_out[' '.join(new_symbols)] = freq
    return v_out

In [24]:
# get_word_char_freqsのテスト
test_corpus = {'apple': 5, 'banana': 3, 'orange': 2}
word_char_freqs = get_word_char_freqs(test_corpus)
print("Word-Character Frequencies:")
for word, freq in word_char_freqs.items():
    print(f"{word}: {freq}")
# get_statsのテスト
stats = get_stats(word_char_freqs)
print("\nAdjacent Pairs Frequencies:")
for pair, freq in stats.items():
    print(f"{pair}: {freq}")
# merge_vocabのテスト
pair_to_merge = ('a', 'n')
merged_vocab = merge_vocab(pair_to_merge, word_char_freqs)
print("\nMerged Vocabulary:")
for word, freq in merged_vocab.items():
    print(f"{word}: {freq}")

Word-Character Frequencies:
a p p l e </w>: 5
b a n a n a </w>: 3
o r a n g e </w>: 2

Adjacent Pairs Frequencies:
('a', 'p'): 5
('p', 'p'): 5
('p', 'l'): 5
('l', 'e'): 5
('e', '</w>'): 7
('b', 'a'): 3
('a', 'n'): 8
('n', 'a'): 6
('a', '</w>'): 3
('o', 'r'): 2
('r', 'a'): 2
('n', 'g'): 2
('g', 'e'): 2

Merged Vocabulary:
a p p l e </w>: 5
b an an a </w>: 3
o r an g e </w>: 2


In [25]:
# BPE学習の簡易シミュレーション
print("--- BPE 学習シミュレーション ---")
# 初期コーパス (単語と頻度)
corpus_example = {"low": 5, "lower": 2, "newest": 6, "widest": 3}
# 1. 文字シーケンスと頻度に変換
vocab_bpe_learn = get_word_char_freqs(corpus_example)
print("初期Vocab (文字分割 + </w>):")
for k,v in vocab_bpe_learn.items(): print(f"  '{k}': {v}")

num_merges = 10 # マージ操作の回数
bpe_merges_learned = {} # 学習されたマージ操作 (ペア -> 優先度)

for i in range(num_merges):
    pairs = get_stats(vocab_bpe_learn)
    if not pairs:
        print("ペアがなくなりました。")
        break
    # 最も頻繁なペアを選択
    best_pair = max(pairs, key=pairs.get)
    
    # マージ操作を保存 (優先度としてマージ順を保存)
    bpe_merges_learned[best_pair] = i 
    
    print(f"Merge {i+1}: {best_pair} (freq: {pairs[best_pair]}) -> {''.join(best_pair)}")
    vocab_bpe_learn = merge_vocab(best_pair, vocab_bpe_learn)
    # print("  Updated Vocab:")
    # for k,v in vocab_bpe_learn.items(): print(f"    '{k}': {v}")

print("\n学習されたBPEマージ操作 (優先度順):")
sorted_merges = sorted(bpe_merges_learned.items(), key=lambda item: item[1])
for pair, priority in sorted_merges:
    print(f"  {pair} -> {''.join(pair)}")

# BPEセグメンテーションの簡易シミュレーション
def segment_bpe(word, learned_merges_sorted_by_priority):
    """ 学習されたマージ操作を使って単語をBPEセグメントに分割 """
    # まず文字に分割 + </w>
    symbols = list(word) + ['</w>']
    
    # 優先度の高いマージ操作から順に適用していく
    # 実際には、最も頻繁なペアがなくなるまで、あるいは現在のセグメントに対して
    # 適用可能な最も優先度の高いマージ操作を繰り返し適用する
    
    # ここでは非常に単純化:
    # 優先度順にマージルールを適用していく。
    # より正しいのは、現在のシンボル列に対して、学習済みマージ操作リストの中から
    # 適用可能な最も優先度の高い（最も早く学習された）ペアを繰り返しマージすること。
    
    # 今回は、学習されたマージ操作のリストを「置き換えルール」として使う
    # (これはBPEの真のセグメンテーションとは異なるが、概念を示すため)
    
    # より論文に近いセグメンテーション:
    # 1. 単語を文字のシーケンスにする: 'l o w e r </w>'
    # 2. 学習済みのマージ操作のリスト（優先度順）を取得
    # 3. シーケンス内で、マージ操作リストに含まれる最も優先度の高いペアを見つけてマージ
    # 4. マージできるペアがなくなるまで繰り返す
    
    word_str_with_spaces = ' '.join(symbols)
    # print(f"Initial segmentation for '{word}': '{word_str_with_spaces}'")

    # 優先度の高いマージから順に適用 (実際にはもっと効率的な方法がある)
    # このループはBPEの正しいセグメンテーションではない。
    # 正しくは、現在の状態から適用可能な最も良いマージを選ぶ。
    # 下記は「学習されたマージ操作を使って新しい単語をどう処理するか」のイメージ。

    # 正しいBPEセグメンテーションの考え方：
    # 1. 単語を文字のリストにする。
    # 2. 現在の文字リストから隣接ペアの頻度を数える（実際は学習済みマージリストを見る）。
    # 3. 学習済みマージリストの中で、現在の文字リストに存在する最も優先度の高いペアをマージする。
    # 4. マージできなくなるまで繰り返す。
    
    current_segments = list(word) + ['</w>']
    
    while True:
        min_priority_found = float('inf')
        best_pair_to_merge_idx = -1
        
        # 現在のセグメント列でマージ可能な最も優先度の高いペアを探す
        for k in range(len(current_segments) - 1):
            pair = (current_segments[k], current_segments[k+1])
            if pair in bpe_merges_learned:
                priority = bpe_merges_learned[pair]
                if priority < min_priority_found:
                    min_priority_found = priority
                    best_pair_to_merge_idx = k
        
        if best_pair_to_merge_idx != -1: # マージ可能なペアが見つかった
            merged_segment = current_segments[best_pair_to_merge_idx] + current_segments[best_pair_to_merge_idx+1]
            current_segments = current_segments[:best_pair_to_merge_idx] + \
                               [merged_segment] + \
                               current_segments[best_pair_to_merge_idx+2:]
            # print(f"  Merged to: {current_segments}")
        else: # これ以上マージできない
            break
            
    return current_segments


print("\n--- BPE セグメンテーションテスト ---")
# 学習されたマージ操作 (タプルキー -> 優先度)
learned_merges_dict = {pair: prio for pair, prio in sorted_merges}

test_word1 = "lowest"
segments1 = segment_bpe(test_word1, learned_merges_dict)
print(f"Word: '{test_word1}', BPE Segments: {segments1}") # 例: ['low', 'est</w>'] や ['lowe', 'st</w>'] など

test_word2 = "newer" # 訓練データにない
segments2 = segment_bpe(test_word2, learned_merges_dict)
print(f"Word: '{test_word2}', BPE Segments: {segments2}") # 例: ['new', 'er</w>']

test_word3 = "bottom" # 語彙にない文字が含まれる可能性 (今回はなし)
segments3 = segment_bpe(test_word3, learned_merges_dict)
print(f"Word: '{test_word3}', BPE Segments: {segments3}")

--- BPE 学習シミュレーション ---
初期Vocab (文字分割 + </w>):
  'l o w </w>': 5
  'l o w e r </w>': 2
  'n e w e s t </w>': 6
  'w i d e s t </w>': 3
Merge 1: ('e', 's') (freq: 9) -> es
Merge 2: ('es', 't') (freq: 9) -> est
Merge 3: ('est', '</w>') (freq: 9) -> est</w>
Merge 4: ('l', 'o') (freq: 7) -> lo
Merge 5: ('lo', 'w') (freq: 7) -> low
Merge 6: ('n', 'e') (freq: 6) -> ne
Merge 7: ('ne', 'w') (freq: 6) -> new
Merge 8: ('new', 'est</w>') (freq: 6) -> newest</w>
Merge 9: ('low', '</w>') (freq: 5) -> low</w>
Merge 10: ('w', 'i') (freq: 3) -> wi

学習されたBPEマージ操作 (優先度順):
  ('e', 's') -> es
  ('es', 't') -> est
  ('est', '</w>') -> est</w>
  ('l', 'o') -> lo
  ('lo', 'w') -> low
  ('n', 'e') -> ne
  ('ne', 'w') -> new
  ('new', 'est</w>') -> newest</w>
  ('low', '</w>') -> low</w>
  ('w', 'i') -> wi

--- BPE セグメンテーションテスト ---
Word: 'lowest', BPE Segments: ['low', 'est</w>']
Word: 'newer', BPE Segments: ['new', 'e', 'r', '</w>']
Word: 'bottom', BPE Segments: ['b', 'o', 't', 't', 'o', 'm', '</w>']


### 3.3 BPEの長所と短所

*   **長所:**
    *   シンプルで効果的なアルゴリズム。
    *   データ駆動でサブワード語彙を自動的に学習できる。
    *   語彙サイズを柔軟に制御できる（マージ回数で調整）。
    *   未知語に対しても、文字レベルまで分解することで何らかの表現を与えることができる。
*   **短所:**
    *   **貪欲アルゴリズム:** 各ステップで最も頻繁なペアをマージするため、必ずしも全体として最適なサブワード分割が得られるとは限らない。
    *   **意味的な考慮の欠如:** 純粋にペアの出現頻度に基づいてマージするため、意味的に不自然なサブワードが生成されることがある（例: "ing" と "ly" が別々に学習されても、"ingly" が一つの単位として学習されるとは限らない）。
    *   **プリトークナイゼーションへの依存:** 元のBPE論文（NLP応用版）では、まずテキストを単語に分割し、その単語の境界を越えないようにBPEを適用することが多い。これは言語依存のプリトークナイザが必要になることを意味する。

## 4. WordPiece (Schuster & Nakajima, 2012; Wu et al., 2016)

WordPieceは、Googleのニューラル機械翻訳システム (GNMT) やBERTで採用されているサブワード分割アルゴリズムです。
基本的な考え方はBPEと似ていますが、ペアをマージする基準が異なります。

*   **BPEとの主な違い:**
    *   **マージ基準:** BPEが最も**出現頻度の高い**隣接ペアをマージするのに対し、WordPieceは、マージすることで訓練データの**尤度 (Likelihood)** を最も増加させるペアをマージします。
    *   **初期語彙:** WordPieceも初期語彙として全ての文字を含みます。
    *   **処理:**
        1.  初期語彙と訓練データで言語モデルを構築。
        2.  現在の語彙を使って、訓練データを最も尤もらしく表現できるペアを見つけ、そのペアを新しいサブワードとして語彙に追加する。
        3.  指定された語彙サイズに達するまで繰り返す。

*   **利点:**
    *   言語モデルの尤度を最大化するようにサブワードが学習されるため、より言語的に自然で、下流タスクの性能向上に繋がりやすいと考えられています。
*   **実装:**
    *   WordPieceの学習アルゴリズムはBPEよりも複雑で、通常はGoogleが提供するツールや、Hugging Face Tokenizersライブラリなどに含まれる実装が利用されます。
    *   スクラッチでの完全な実装は、このノートブックの範囲を超えます。

WordPieceは、単語の先頭に特別なプレフィックス（例: `##`）を付けて、それが単語の途中から始まるサブワードであることを示すことが多いです（例: "playing" -> `["play", "##ing"]`）。

## 5. SentencePiece (Kudo & Richardson, 2018)

SentencePieceは、GoogleのTaku Kudo氏らによって開発された、言語非依存のサブワード分割器および脱トークン化器です。
BPEやWordPieceが抱えるいくつかの課題、特にプリトークナイゼーションへの依存を解決することを目指しています。

### 5.1 SentencePieceの主要な特徴と設計思想

*   **言語非依存性 (No Pre-tokenization):**
    *   SentencePieceは、**生の文（raw sentences）から直接サブワードモデルを学習します。** 事前にテキストを単語に分割する（プリトークナイズする）必要がありません。
    *   これにより、空白で単語を区切らない言語（日本語、中国語、タイ語など）や、複雑な複合語を持つ言語（ドイツ語、フィンランド語など）にも統一的に適用できます。
*   **可逆的なトークナイゼーション (Lossless Tokenization):**
    *   SentencePieceは、トークン化されたサブワードシーケンスから元の正規化された文を**完全に復元**できるように設計されています。
    *   これを実現するために、**空白文字も通常の文字と同様に扱い**、必要であれば特別なメタシンボル（デフォルトでは ` ` (U+2581) という記号で空白を表す）に置き換えてからサブワード分割を行います。
    *   デトークナイズ（脱トークン化）は、単純にサブワードを結合し、メタシンボル ` ` を実際の空白文字に戻すだけで完了します。
*   **2つのサブワード分割アルゴリズムをサポート:**
    *   **BPE:** Sennrichらのアルゴリズムに基づいています。
    *   **Unigram Language Model Tokenizer (Kudo, 2018):**
        *   全ての可能なサブワード分割に対して、それぞれの分割の確率（ユニグラム言語モデルで計算）を求め、全体の尤度が最大になるような分割を選択します。
        *   学習時には、まず非常に大きなサブワード候補の集合から始め、期待される損失増加が最も小さいサブワードを徐々に削除していくことで、最終的な語彙サイズに収めます。
*   **自己完結型のモデルファイル:**
    *   学習されたサブワードモデル（語彙、マージルール、正規化ルールなど）は、単一のモデルファイルに保存されます。これにより、再現性が高く、モデルの配布も容易になります。
*   **サブワード正則化 (Subword Regularization) - オプション:**
    *   SentencePieceは、訓練時に複数の可能なサブワード分割を確率的にサンプリングする「サブワード正則化」というテクニックをサポートしています（主にUnigram LM Tokenizerの場合）。これにより、モデルの頑健性が向上し、過学習を抑制する効果が期待されます。

### 5.2 SentencePieceライブラリの基本的な使い方
ここでは、Pythonライブラリ `sentencepiece` を使った簡単な例を示します。
まず、`pip install sentencepiece` でライブラリをインストールしてください。

In [28]:
print("--- SentencePiece ライブラリテスト ---")
    
# 0. ダミーの訓練データファイルを作成 (1行1文のテキストファイル)
dummy_corpus_file = "dummy_corpus_for_spm.txt"
with open(dummy_corpus_file, "w", encoding="utf-8") as f:
    f.write("This is the first sentence for SentencePiece.\n")
    f.write("SentencePiece can handle raw text directly.\n")
    f.write("こんにちは世界。日本語も扱えます。\n") # 日本語の例
    f.write("New York is a big city.\n")
    f.write("I love natural language processing and subword tokenization.\n")

# 1. SentencePieceモデルの学習
# --input: 訓練データファイル
# --model_prefix: 出力されるモデルファイル名のプレフィックス (例: my_sp_model)
#                 (my_sp_model.model と my_sp_model.vocab が生成される)
# --vocab_size: 最終的な語彙サイズ
# --model_type: bpe または unigram (デフォルト), char, word
# --character_coverage: 基本文字セットのカバー率 (デフォルト0.9995)
# --bos_id=1, --eos_id=2, --unk_id=0, --pad_id=-1 (デフォルトは-1だが、モデルによって調整)
    
model_prefix = 'my_sp_model'
spm.SentencePieceTrainer.train(
    f'--input={dummy_corpus_file} --model_prefix={model_prefix} '
    f'--vocab_size=300 --model_type=bpe '
    f'--bos_id=1 --eos_id=2 --unk_id=0 --pad_id=-1 ' # PyTorchのEmbeddingはpad_id=0を期待することが多いので注意
    f'--character_coverage=1.0' # 全ての文字をカバー
)
print(f"\nSentencePiece BPEモデル '{model_prefix}.model' の学習完了。")

# 学習済みモデルのロード
sp = spm.SentencePieceProcessor()
sp.load(f'{model_prefix}.model')
print("学習済みモデルのロード完了。")

# 2. テキストのエンコード (サブワード分割 + ID化)
text1 = "This is a test for SentencePiece."
ids1 = sp.encode_as_ids(text1)
pieces1 = sp.encode_as_pieces(text1)
print(f"\nText: '{text1}'")
print(f"  Pieces: {pieces1}")
print(f"  IDs: {ids1}")

text2 = "ニューヨークは大きな都市です。" # 日本語の例
ids2 = sp.encode_as_ids(text2)
pieces2 = sp.encode_as_pieces(text2)
print(f"\nText: '{text2}'")
print(f"  Pieces: {pieces2}")
print(f"  IDs: {ids2}")
        
# 未知語の扱い (語彙に含まれない文字シーケンス)
text_oov = "NewUnseenWord" # 語彙にない可能性が高い
pieces_oov = sp.encode_as_pieces(text_oov)
ids_oov = sp.encode_as_ids(text_oov)
print(f"\nOOV Text: '{text_oov}'")
print(f"  Pieces: {pieces_oov}") # 文字単位か、<unk> になるか (char_coverageによる)
print(f"  IDs: {ids_oov}")


# 3. IDシーケンスのデコード (元のテキストに戻す)
decoded_text1 = sp.decode_ids(ids1)
decoded_text2 = sp.decode_ids(ids2)
print(f"\nDecoded IDs {ids1} -> '{decoded_text1}'")
print(f"Decoded IDs {ids2} -> '{decoded_text2}'")
        
# 4. サブワード正則化 (N-best segmentation / sampling)
#    model_type=unigram で学習した場合に特に効果的
#    sp.sample_encode_as_pieces() や sp.sample_encode_as_ids()
print("\nSubword Regularization (Sampling for BPE - 概念):")
for _ in range(3): # 3回サンプリング
    # BPEモデルではsample_encodeは決定論的だが、unigramなら確率的になる
    sampled_pieces = sp.encode_as_pieces(text1, enable_sampling=True, alpha=0.1, nbest_size=-1)
    print(f"  Sampled pieces for '{text1}': {sampled_pieces}")

import os
if os.path.exists(dummy_corpus_file): os.remove(dummy_corpus_file)
if os.path.exists(f"{model_prefix}.model"): os.remove(f"{model_prefix}.model")
if os.path.exists(f"{model_prefix}.vocab"): os.remove(f"{model_prefix}.vocab")

--- SentencePiece ライブラリテスト ---

SentencePiece BPEモデル 'my_sp_model.model' の学習完了。
学習済みモデルのロード完了。

Text: 'This is a test for SentencePiece.'
  Pieces: ['▁This', '▁is', '▁a', '▁te', 'st', '▁for', '▁SentencePiece', '.']
  IDs: [98, 23, 49, 221, 46, 88, 26, 264]

Text: 'ニューヨークは大きな都市です。'
  Pieces: ['▁', 'ニューヨーク', 'は', '大きな都市で', 'す', '。']
  IDs: [253, 0, 290, 0, 287, 276]

OOV Text: 'NewUnseenWord'
  Pieces: ['▁New', 'U', 'n', 'se', 'en', 'W', 'ord']
  IDs: [83, 0, 255, 155, 3, 0, 70]

Decoded IDs [98, 23, 49, 221, 46, 88, 26, 264] -> 'This is a test for SentencePiece.'
Decoded IDs [253, 0, 290, 0, 287, 276] -> ' ⁇ は ⁇ す。'

Subword Regularization (Sampling for BPE - 概念):
  Sampled pieces for 'This is a test for SentencePiece.': ['▁This', '▁is', '▁a', '▁te', 'st', '▁f', 'or', '▁S', 'ent', 'en', 'ce', 'Piece', '.']
  Sampled pieces for 'This is a test for SentencePiece.': ['▁This', '▁is', '▁a', '▁te', 'st', '▁f', 'o', 'r', '▁S', 'ent', 'ence', 'Piece', '.']
  Sampled pieces for 'This is a test f

## 6. 各サブワード分割手法の比較と考察

| 特徴                       | BPE (Byte Pair Encoding)                                  | WordPiece                                                   | SentencePiece                                                                 |
| :------------------------- | :-------------------------------------------------------- | :---------------------------------------------------------- | :---------------------------------------------------------------------------- |
| **基本アイデア**             | 最も頻繁な隣接ペアをマージ                                  | マージで言語モデル尤度が最大になるペアをマージ                  | 生のテキストから直接学習。空白も文字として扱う。BPEまたはUnigram LMを選択可能。 |
| **プリトークナイゼーション** | 通常必要（単語境界を越えないように）                        | 通常必要                                                      | **不要（言語非依存）**                                                        |
| **語彙生成**             | データ駆動（マージ回数で語彙サイズ制御）                   | データ駆動（尤度基準で語彙サイズ制御）                        | データ駆動（指定した語彙サイズになるように学習）                                |
| **未知語(OOV)対応**        | 文字レベルまで分解可能                                      | 文字レベルまで分解可能、または特別なサブワード                  | 文字レベルまで分解可能、または`<unk>`トークン                                    |
| **可逆性**                 | プリトークナイザとBPEモデルに依存（工夫すれば可逆にも）         | プリトークナイザとモデルに依存                                  | **完全に可逆（Lossless）**                                                    |
| **主な利用例**             | 初期のNMT、GPT-2など                                     | BERT、GoogleのNMT (GNMT)など                               | Googleの多くのNLPモデル、多言語モデル、T5など                                   |
| **トークン化の仕方**       | ` ` (スペース) で単語を区切り、単語内でBPEを適用。単語末尾に`</w>`など。 | 単語内で分割し、継続サブワードに`##`プレフィックス。          | 空白を`_` (U+2581) に置き換え、文全体をサブワードに分割。                       |
| **実装**                   | 比較的シンプル                                            | BPEより複雑（尤度計算が絡む）                                 | C++ライブラリとして提供。Python APIあり。                                       |

**考察:**
*   **BPE** はシンプルで理解しやすく、多くの場面で効果的なサブワード分割手法です。
*   **WordPiece** は言語モデルの尤度を考慮するため、より言語的に自然な分割が得られると期待されますが、アルゴリズムは若干複雑です。
*   **SentencePiece** は、プリトークナイゼーションが不要で、言語非依存的に扱えるという大きな利点があります。また、Unigram Language Modelベースの分割やサブワード正則化といった高度な機能も提供しており、現代の多くの最先端モデルで採用されています。可逆性も保証されているため、デトークナイズが容易です。

どの手法を選択するかは、タスクの要件、対象言語、利用可能なツール、計算資源などによって異なります。
現代の多言語対応やEnd-to-End学習の流れを考えると、SentencePieceのような言語非依存的で自己完結型のツールは非常に強力です。